Script to batch process a Sentinel-1a TOPS mode data set and create time series.


***Curently not working - use readme files instead***
  
Xiaohua(Eric) Xu  
June 2016  
Adapted to Jupyter September 5, 2023

First, prepare the files

In [5]:
import os

In [ ]:
!rm -rf raw
!mkdir raw
os.chdir("raw")

In order to correct for Elevation Antenna Pattern Change, cat the manifest and aux files to the xmls and
delete the first line of the manifest file as it's not a typical xml file.

In [ ]:
!awk 'NR>1 {print $0}' < ../raw_orig/20150121_manifest.safe > tmp_file
!cat ../raw_orig/s1a-iw1-slc-vv-20150121t134413-20150121t134424-004270-005317-001.xml tmp_file ../raw_orig/s1a-aux-cal.xml > ./s1a-iw1-slc-vv-20150121t134413-20150121t134424-004270-005317-001.xml
!awk 'NR>1 {print $0}' < ../raw_orig/20150310_manifest.safe > tmp_file
!cat ../raw_orig/s1a-iw1-slc-vv-20150310t134412-20150310t134424-004970-006386-001.xml tmp_file ../raw_orig/s1a-aux-cal.xml > ./s1a-iw1-slc-vv-20150310t134412-20150310t134424-004970-006386-001.xml
!awk 'NR>1 {print $0}' < ../raw_orig/20150403_manifest.safe > tmp_file
!cat ../raw_orig/s1a-iw1-slc-vv-20150403t134413-20150403t134424-005320-006bc4-001.xml tmp_file ../raw_orig/s1a-aux-cal.xml > ./s1a-iw1-slc-vv-20150403t134413-20150403t134424-005320-006bc4-001.xml
!awk 'NR>1 {print $0}' < ../raw_orig/20150427_manifest.safe > tmp_file
!cat ../raw_orig/s1a-iw1-slc-vv-20150427t134414-20150427t134426-005670-00745c-001.xml tmp_file ../raw_orig/s1a-aux-cal.xml > ./s1a-iw1-slc-vv-20150427t134414-20150427t134426-005670-00745c-001.xml
!awk 'NR>1 {print $0}' < ../raw_orig/20150521_manifest.safe > tmp_file
!cat ../raw_orig/s1a-iw1-slc-vv-20150521t134415-20150521t134427-006020-007c3f-001.xml tmp_file ../raw_orig/s1a-aux-cal.xml > ./s1a-iw1-slc-vv-20150521t134415-20150521t134427-006020-007c3f-001.xml
!rm tmp_file
!ln -s ../raw_orig/*EOF .
!ln -s ../raw_orig/*tiff .
!ln -s ../topo/dem.grd .

Start batch preprocessing

In [ ]:
!echo "s1a-iw1-slc-vv-20150403t134413-20150403t134424-005320-006bc4-001:S1A_OPER_AUX_POEORB_OPOD_20150424T123026_V20150402T225944_20150404T005944.EOF" > data.in
!echo "s1a-iw1-slc-vv-20150121t134413-20150121t134424-004270-005317-001:S1A_OPER_AUX_POEORB_OPOD_20150211T122339_V20150120T225944_20150122T005944.EOF" >> data.in
!echo "s1a-iw1-slc-vv-20150310t134412-20150310t134424-004970-006386-001:S1A_OPER_AUX_POEORB_OPOD_20150331T123006_V20150309T225944_20150311T005944.EOF" >> data.in
!echo "s1a-iw1-slc-vv-20150427t134414-20150427t134426-005670-00745c-001:S1A_OPER_AUX_POEORB_OPOD_20150518T122528_V20150426T225944_20150428T005944.EOF" >> data.in
!echo "s1a-iw1-slc-vv-20150521t134415-20150521t134427-006020-007c3f-001:S1A_OPER_AUX_POEORB_OPOD_20150611T122710_V20150520T225944_20150522T005944.EOF" >> data.in

Get the baseline_time plot first, select the supermaster and mv it to the first line in data.in, save the baseline_table.dat for sbas use.

In [ ]:
!preproc_batch_tops.csh data.in dem.grd 2
os.chdir("..")

In case there are clear burst boundaries, use *preproc_batch_tops_esd.csh* instead

Create the files needed for intf_tops.csh

In [ ]:
!rm -f intf.in
!rm -r intf intf_all
!echo "S1_20150121_ALL_F1:S1_20150310_ALL_F1" > intf.in
!echo "S1_20150121_ALL_F1:S1_20150403_ALL_F1" >> intf.in
!echo "S1_20150121_ALL_F1:S1_20150427_ALL_F1" >> intf.in
!echo "S1_20150310_ALL_F1:S1_20150403_ALL_F1" >> intf.in
!echo "S1_20150310_ALL_F1:S1_20150427_ALL_F1" >> intf.in
!echo "S1_20150310_ALL_F1:S1_20150521_ALL_F1" >> intf.in
!echo "S1_20150403_ALL_F1:S1_20150427_ALL_F1" >> intf.in
!echo "S1_20150403_ALL_F1:S1_20150521_ALL_F1" >> intf.in
!echo "S1_20150427_ALL_F1:S1_20150521_ALL_F1" >> intf.in

Process the interferograms, remember to set your super master in the batch_tops.config file.

In [ ]:
!intf_tops.csh intf.in batch_tops.config

After this is done, check the interferograms. You'll find there being a discontinuity in some of the interferograms. 
This can be resolved by using `preproc_batch_tops_esd.csh`. Modify the `README_prep.csh` and see what happens   
Also note the ESD may not always make your results better.

Prepare the input files needed for sbas

In [10]:
os.chdir("..")

In [11]:
!rm -fr SBAS
!mkdir SBAS
os.chdir("SBAS")
!pwd
#
# based on baseline_table.dat create the intf.tab and scene.tab for sbas
#
# phase  corherence  ref_id  rep_id  baseline
!echo "../intf_all/2015020_2015068/unwrap.grd ../intf_all/2015020_2015068/corr.grd 2015020 2015068 -90.333" > intf.tab
!echo "../intf_all/2015020_2015092/unwrap.grd ../intf_all/2015020_2015092/corr.grd 2015020 2015092 -52.6118" >> intf.tab
!echo "../intf_all/2015020_2015116/unwrap.grd ../intf_all/2015020_2015116/corr.grd 2015020 2015116 -83.0324" >> intf.tab
!echo "../intf_all/2015068_2015092/unwrap.grd ../intf_all/2015068_2015092/corr.grd 2015068 2015092 37.7213" >> intf.tab
!echo "../intf_all/2015068_2015116/unwrap.grd ../intf_all/2015068_2015116/corr.grd 2015068 2015116 7.30062" >> intf.tab
!echo "../intf_all/2015068_2015140/unwrap.grd ../intf_all/2015068_2015140/corr.grd 2015068 2015140 -13.2457" >> intf.tab
!echo "../intf_all/2015092_2015116/unwrap.grd ../intf_all/2015092_2015116/corr.grd 2015092 2015116 -30.4207" >> intf.tab
!echo "../intf_all/2015092_2015140/unwrap.grd ../intf_all/2015092_2015140/corr.grd 2015092 2015140 -50.967" >> intf.tab
!echo "../intf_all/2015116_2015140/unwrap.grd ../intf_all/2015116_2015140/corr.grd 2015116 2015140 -20.5463" >> intf.tab
#
# scene_id  day
!echo "2015092 448" > scene.tab
!echo "2015020 376" >> scene.tab 
!echo "2015068 424" >> scene.tab
!echo "2015116 472" >> scene.tab
!echo "2015140 496" >> scene.tab
!xdim=`gmt grdinfo -C ../intf_all/2015068_2015092/unwrap.grd | awk '{print $10}'`
!ydim=`gmt grdinfo -C ../intf_all/2015068_2015092/unwrap.grd | awk '{print $11}'`
#
# run sbas
#
!sbas intf.tab scene.tab 9 5 "$xdim" "$ydim" -smooth 1.0 -wavelength 0.0554658 -incidence 30 -range 800184.946186 -rms -dem
#
# project the velocity to Geocooridnates
#
!ln -s ../topo/trans.dat .
!proj_ra2ll.csh trans.dat vel.grd vel_ll.grd
!gmt grd2cpt vel_ll.grd -T= -Z -Cjet > vel_ll.cpt
!grd2kml.csh vel_ll vel_ll.cpt

!cd ..


/Users/martinhawks/.Trash/SBAS
grdinfo [ERROR]: Cannot find file ../intf_all/2015068_2015092/unwrap.grd
grdinfo [ERROR]: Must specify one or more input files
grdinfo [ERROR]: Cannot find file ../intf_all/2015068_2015092/unwrap.grd
grdinfo [ERROR]: Must specify one or more input files

compute RMS misfit
compute DEM error

setting smoothing to   1.000 
setting radar wavelength to   0.055 m 
setting radar incidence angle to  30.000 degree 
setting range to 800184.946 m 
Required Memory Usage is 0.000013 GB ...
Memory Allocation Successful...
read table file ...
number of SAR scenes is 5 
number of interferograms is 9 
read phase and correlation grids ...
sbas [ERROR]: Cannot find file ../intf_all/2015020_2015068/corr.grd
sbas [ERROR]: Cannot find file ../intf_all/2015020_2015068/corr.grd
sbas [ERROR]: File ../intf_all/2015020_2015068/corr.grd not found
[Session sbas (0)]: Error returned from GMT API: GMT_FILE_NOT_FOUND (16)
 Can't open  ../intf_all/2015020_2015068/corr.grd 
proj_ra2ll.cs

In [9]:
!pwd
!ls ../intf_all/2015020_2015068/corr.grd

/Users/martinhawks/SchoolDocs/SAR_Prossesing/Jupyter-Main/Jupyter-Adaptation/test_GMTSAR/S1A_Stack_CPGF_T173_sbas_ready/SBAS/SBAS


ls: ../intf_all/2015020_2015068/corr.grd: No such file or directory


Based on `baseline_table.dat` create the `intf.tab` and `scene.tab` for sbas

In [ ]:
#     phase                                   corherence                           ref_id  rep_id  baseline
!echo "../intf_all/2015020_2015068/unwrap.grd ../intf_all/2015020_2015068/corr.grd 2015020 2015068 -90.333" > intf.tab
!echo "../intf_all/2015020_2015092/unwrap.grd ../intf_all/2015020_2015092/corr.grd 2015020 2015092 -52.6118" >> intf.tab
!echo "../intf_all/2015020_2015116/unwrap.grd ../intf_all/2015020_2015116/corr.grd 2015020 2015116 -83.0324" >> intf.tab
!echo "../intf_all/2015068_2015092/unwrap.grd ../intf_all/2015068_2015092/corr.grd 2015068 2015092 37.7213" >> intf.tab
!echo "../intf_all/2015068_2015116/unwrap.grd ../intf_all/2015068_2015116/corr.grd 2015068 2015116 7.30062" >> intf.tab
!echo "../intf_all/2015068_2015140/unwrap.grd ../intf_all/2015068_2015140/corr.grd 2015068 2015140 -13.2457" >> intf.tab
!echo "../intf_all/2015092_2015116/unwrap.grd ../intf_all/2015092_2015116/corr.grd 2015092 2015116 -30.4207" >> intf.tab
!echo "../intf_all/2015092_2015140/unwrap.grd ../intf_all/2015092_2015140/corr.grd 2015092 2015140 -50.967" >> intf.tab
!echo "../intf_all/2015116_2015140/unwrap.grd ../intf_all/2015116_2015140/corr.grd 2015116 2015140 -20.5463" >> intf.tab

In [ ]:
#     scene_id day
!echo "2015092 448" > scene.tab
!echo "2015020 376" >> scene.tab 
!echo "2015068 424" >> scene.tab
!echo "2015116 472" >> scene.tab
!echo "2015140 496" >> scene.tab

Run sbas

In [ ]:
!xdim=`gmt grdinfo -C ../intf_all/2015068_2015092/unwrap.grd | awk '{print $10}'`
!ydim=`gmt grdinfo -C ../intf_all/2015068_2015092/unwrap.grd | awk '{print $11}'`
!sbas intf.tab scene.tab 9 5 "$xdim" "$ydim" -smooth 1.0 -wavelength 0.0554658 -incidence 30 -range 800184.946186 -rms -dem

Project the velocity to Geocooridnates

In [ ]:
!ln -s ../topo/trans.dat .
!proj_ra2ll.csh trans.dat vel.grd vel_ll.grd
!gmt grd2cpt vel_ll.grd -T= -Z -Cjet > vel_ll.cpt
!grd2kml.csh vel_ll vel_ll.cpt

os.chdir("..")